# Tahmin Metotları için kullanılabilecek Python Şablonu

# Kütüphaneler

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
import statsmodels.api as sm

# numpy ve pandas kütüphanelerini veriyi işlemek ve hafızada yönlendirmek için kullanıyoruz (data frame gibi sınıflar için)

# Veri Yükleme

In [2]:
# veri yukleme

veriler = pd.read_csv('veriler/maaslar_yeni.csv')
x = veriler.iloc[:,2:5]
y = veriler.iloc[:,5:]
X = x.values
Y = y.values

# Veri yükleme aşamasında, verinin yükleneceği dosyanın yanında, bağımlı ve bağımsız değişkenleri içeren iki ayrı kolon oluşturulmalıdır. Bağımsız değişkenlerin tamamının x isminde bir dataframe içerisinde ve tek bir kolondan oluşan bağımlı değişkenleri ise y ismine bir data firma içerisinde duracağı kabulü yapılmıştır. Ayrıca X ve Y değişkenleri de numpy dizisi olarak bu dataframe'lerden .values özelliğini alır. 

# Korelasyon Matrisi

In [3]:
print(veriler.corr()) 

# Veriler üzerinde karar verirken, kullanılacak önemli ön işleme aşamalarından birsi de korelasyon matrisidir ve bu matris ile kolonların birbiri ile olan ilişkisi görülebilir. 

               Calisan ID  UnvanSeviyesi     Kidem      Puan      maas
Calisan ID       1.000000       0.331847  0.206278 -0.251278  0.226287
UnvanSeviyesi    0.331847       1.000000 -0.125200  0.034948  0.727036
Kidem            0.206278      -0.125200  1.000000  0.322796  0.117964
Puan            -0.251278       0.034948  0.322796  1.000000  0.201474
maas             0.226287       0.727036  0.117964  0.201474  1.000000


# Linear Regression

In [4]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X,Y)
model = sm.OLS(lin_reg.predict(X),X)
print(model.fit().summary())
print("Linear R2 degeri:")
print(r2_score(Y, lin_reg.predict((X))))

# Sci-Kit Learn kütüphanesinin genel bir özelliği, fit() fonksiyonu ile eğitmesi ve predict fonksiyonu ile tahminde bulunmasıdır. Buradaki örnekte doğrusal regresyon (linear regression) üzerinden X ve Y dizileri verilerek bir makine öğrenmesi algoritması eğitilmiş ve oluşan model daha sonra OLS ve r2_score fonksiyonları ile ölçülmüştür. Ölçüm sırasında gerçek verileri tutan Y değişkeni (numpy dizisi) ile lin_reg objesinin predict fonksiyonun döndürdüğü tahmin değerleri (yani modelin tahmin etttiği değerler) karşılaştırılmış dolayısıyla modelin ne kadar başarılı tahmin yaptığı ölçülmüştür.

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.903
Model:                            OLS   Adj. R-squared (uncentered):              0.892
Method:                 Least Squares   F-statistic:                              83.89
Date:                Sun, 18 Oct 2020   Prob (F-statistic):                    8.38e-14
Time:                        17:25:43   Log-Likelihood:                         -295.74
No. Observations:                  30   AIC:                                      597.5
Df Residuals:                      27   BIC:                                      601.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# Polynomial Regression

In [5]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 4)
x_poly = poly_reg.fit_transform(X)
print(x_poly)
lin_reg2 = LinearRegression()
lin_reg2.fit(x_poly,y)
model2 = sm.OLS(lin_reg2.predict(poly_reg.fit_transform(X)),X)
print(r2_score(Y, lin_reg2.predict(poly_reg.fit_transform(X)) ))

# Polynomial regression yöntemi aslında doğrusal regresyondan farklı değildir. Hatta aynı nesne ve fonksiyonların kullanıldığı söylenebilir. Buradaki hile, verilerin doğrusal regresyona verilmeden önce bir polinom öznitelik fonksiyonuna verilmesidir. bu işlem yukarıdaki kodda da gösterildiği üzere, PolynomialFeattures nesnesi üzerinden yapılmaktadır. 

[[1.0000000e+00 1.0000000e+00 5.0000000e+00 ... 1.2250000e+05
  1.7150000e+06 2.4010000e+07]
 [1.0000000e+00 2.0000000e+00 5.0000000e+00 ... 1.2250000e+05
  1.7150000e+06 2.4010000e+07]
 [1.0000000e+00 3.0000000e+00 5.0000000e+00 ... 1.2250000e+05
  1.7150000e+06 2.4010000e+07]
 ...
 [1.0000000e+00 8.0000000e+00 1.0000000e+01 ... 1.4440000e+05
  5.4872000e+05 2.0851360e+06]
 [1.0000000e+00 9.0000000e+00 1.0000000e+00 ... 2.5000000e+03
  1.2500000e+05 6.2500000e+06]
 [1.0000000e+00 1.0000000e+01 9.0000000e+00 ... 5.5800900e+05
  5.1460830e+06 4.7458321e+07]]
0.9990003560225863


# Destek Vektör Regresyonu ve Ölçekleme (Support Vector Regression , Scaling)

In [6]:
from sklearn.preprocessing import StandardScaler
sc1 = StandardScaler()
x_olcekli = sc1.fit_transform(X)
sc2 = StandardScaler()
y_olcekli = np.ravel(sc2.fit_transform(Y.reshape(-1,1)))
from sklearn.svm import SVR
svr_reg = SVR(kernel = 'rbf')
svr_reg.fit(x_olcekli,y_olcekli)
model3 = sm.OLS(svr_reg.predict(x_olcekli),x_olcekli)
print(model3.fit().summary())
print(r2_score(y_olcekli, svr_reg.predict(x_olcekli)) )

# Destek vektör regresyonunun en önemli özelliği, marjinal verilere karşı hassas olmasıdır. Bu yüzden ve verilerin daha iyi tahminini sağlamak için, öncelikle standartlaştırma yapılması gerekir. Yukarıdaki kodun ilk parçasında, StandardScaler sınıfından türettilmiş sc1 ve sc2 nesneleri sayesinde hem X hem de Y dizileri ölçeklenmektedir. Ardından SVR sınıfından rbf çekirdeği (kernel) ile üretilen svr_reg isimli nesne ile tahmin modeli oluşturulmakta, bunun için de sklearn klasiği olan fit metodu kullanılmaktadır

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.782
Model:                            OLS   Adj. R-squared (uncentered):              0.758
Method:                 Least Squares   F-statistic:                              32.37
Date:                Sun, 18 Oct 2020   Prob (F-statistic):                    4.34e-09
Time:                        17:30:38   Log-Likelihood:                        -0.92453
No. Observations:                  30   AIC:                                      7.849
Df Residuals:                      27   BIC:                                      12.05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# Karar Ağacı ile Tahmin (Decision Tree)

In [7]:
from sklearn.tree import DecisionTreeRegressor
r_dt = DecisionTreeRegressor(random_state=0)
r_dt.fit(X,Y)
print('Decision Tree OLS')
model4 = sm.OLS(r_dt.predict(X),X)
print(model4.fit().summary())
print("Decision Tree R2 degeri:")
print(r2_score(Y, r_dt.predict(X)) )

# Yapı olarak buraya kadar kullanılan sınıflandırma algoritmalarından pek de farklı olmayan karar ağacı sınıflandırması, DecisionTreeRegressor sınıfından türetilmiş ve yine X ve Y dizileri üzerinde fit() metodu ile bir model inşası için kullanılmıştır. 

Decision Tree OLS
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.679
Model:                            OLS   Adj. R-squared (uncentered):              0.644
Method:                 Least Squares   F-statistic:                              19.08
Date:                Sun, 18 Oct 2020   Prob (F-statistic):                    7.62e-07
Time:                        17:32:58   Log-Likelihood:                         -317.95
No. Observations:                  30   AIC:                                      641.9
Df Residuals:                      27   BIC:                                      646.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

# Rassal Orman (Random Forest)

In [9]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators = 10, random_state=0)
rf_reg.fit(X,Y.ravel())
print('Random Forest OLS')
model5 = sm.OLS(rf_reg.predict(X),X)
print(model5.fit().summary())
print("Random Forest R2 degeri:")
print(r2_score(Y, rf_reg.predict(X)) )

# Alt yapısında karar ağacı kullanan rassal ormanlar (random forest), şimdiye kadar elde ettiğimiz yapıya çok benzer şekilde, sklearn kütüphanesi içerisinden bir sınıf olarak RandomForestRegressor eklemiş (import) ve bu sınıftan da ürettiğimiz nesne ile fit() ve predict() metotlarını kullanarak makine öğrenmesini gerçekleştirmiştir. 

Random Forest OLS
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.713
Model:                            OLS   Adj. R-squared (uncentered):              0.681
Method:                 Least Squares   F-statistic:                              22.31
Date:                Sun, 18 Oct 2020   Prob (F-statistic):                    1.79e-07
Time:                        17:34:01   Log-Likelihood:                         -316.07
No. Observations:                  30   AIC:                                      638.1
Df Residuals:                      27   BIC:                                      642.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

## Neler Öğrendik:
1. Sci-Kit Learn kütüphanesinin genel akışını ve fit(), predict() gibi metotların genel kullanımını
2. Doğrusal regresyon, Polinomla regresyon, destek vektör regresyonu, karar ağacı regresyonu ve rassal orman regresyonu metotlarının kullanımını
3. Korrelasyon matrisini ve önemini
4. r2_score ve OLS metotları ile yöntemler arasında karşılaştırma yapmayı
